# Sample notebook for ARCS

This notebook demonstrates how to use the ARCS library to perform a simulation on a chemical reaction network. 

In [34]:
import pandas as pd
from arcs.traversal import traverse
from arcs.analysis import AnalyseSampling

In [42]:
temperature = 300
pressure = 10
concs = {"SO2": 10e-6, "NO2": 50e-6, "H2S": 30e-6, "H2O": 20e-6}

In [43]:
results = traverse(
    temperature,
    pressure,
    concs,
    sample_length=320,
)

progress:           |                                                  |<string>:2: RuntimeWarning:

divide by zero encountered in log

/Users/HAEGG/code/ccs/arcs/.venv/lib/python3.10/site-packages/chempy/equilibria.py:276: UserWarning:

Too much of at least one component

/Users/HAEGG/code/ccs/arcs/.venv/lib/python3.10/site-packages/chempy/equilibria.py:276: UserWarning:

Too much of at least one component

<string>:2: RuntimeWarning:

divide by zero encountered in log

/Users/HAEGG/code/ccs/arcs/.venv/lib/python3.10/site-packages/chempy/equilibria.py:276: UserWarning:

Too much of at least one component

<string>:2: RuntimeWarning:

divide by zero encountered in log

/Users/HAEGG/code/ccs/arcs/.venv/lib/python3.10/site-packages/chempy/equilibria.py:276: UserWarning:

Too much of at least one component

<string>:2: RuntimeWarning:

divide by zero encountered in log

/Users/HAEGG/code/ccs/arcs/.venv/lib/python3.10/site-packages/chempy/equilibria.py:276: UserWarning:

Too much of at lea

In [44]:
df_d = pd.DataFrame(results.initfinaldiff)
display(df_d)

,initial,final,change
SO2,10.0,8.742471,-1.257529
NO2,50.0,45.235471,-4.764529
H2S,30.0,28.734979,-1.265021
H2O,20.0,19.608279,-0.391721
NOHSO4,0.0,2.008610,0.000000
N2,0.0,1.287755,0.000000
H2SO4,0.0,4.272817,0.000000
NO,0.0,5.345540,0.000000
HNO3,0.0,2.722222,0.000000


In [45]:
### statistics
analyse = AnalyseSampling(results.data, markdown=True)

analyse.mean_sampling()
analyse.reaction_paths()
analyse.reaction_statistics()

mean = analyse.mean_data
paths = pd.DataFrame(analyse.common_paths)
stats = pd.DataFrame(analyse.stats)

df_m_t = pd.DataFrame(mean).T
df_m_t = df_m_t[df_m_t["value"] != 0]

df_m = pd.DataFrame(
    {
        "comps": list(df_m_t.T.keys()),
        "values": df_m_t["value"].values,
        "variance": df_m_t["variance"].values,
        "variance_minus": -df_m_t["variance"].values,
    }
)

In [46]:
import plotly.express as px

fig = px.bar(
    df_m,
    x="comps",
    y="values",
    error_y="variance",
    error_y_minus="variance_minus",
    labels={"comps": "", "values": "\u0394 ppm"},
    color="values",
    color_continuous_scale="tropic_r",
    hover_data={
        "values": False,
        "comps": False,
        "variance": False,
        "error": (":.2e", df_m["variance"]),
        "specie": df_m["comps"],
        "PPM": (":.1f", df_m["values"]),
    },
    width=1000,
    height=500,
)

fig.update_xaxes(showgrid=False, tickangle=-60, tickmode="linear")

In [47]:
display(paths)

,paths,k,frequency
0,2NH<sub>3</sub> + 3SO<sub>2</sub> = 3H<sub>2</...,1.43e-172 \n 4.69e+43,5
1,H<sub>2</sub>S + 3H<sub>2</sub>SO<sub>4</sub> ...,1.7e+43 \n 1.43e-172,4
2,2H<sub>2</sub>S + 3O<sub>2</sub> = 2H<sub>2</s...,7.65e+173 \n 1.43e-172,3
3,2NH<sub>3</sub> + 3SO<sub>2</sub> = 3H<sub>2</...,1.43e-172 \n 7.45e-200,2
4,2NH<sub>3</sub> + 3SO<sub>2</sub> = 3H<sub>2</...,1.43e-172 \n 3.62e-129,1


In [48]:
display(stats)

,index,k,frequency
0,2 NH3 + 3 SO2 = 3 H2S + 2 NO2 + O2,1.43e-172,73
1,2 H2S + 3 O2 = 2 H2O + 2 SO2,7.65e+173,55
2,2 NH3 + 2 SO2 = H2 + 2 H2S + 2 NO2,3.62e-129,46
3,H2S + 3 H2SO4 = 4 H2O + 4 SO2,1.7e+43,40
4,3 H2 + SO2 = 2 H2O + H2S,4.69e+43,37
5,N2 + 2 NOHSO4 = H2S + 4 NO2 + SO2,2.25e-134,16
6,2 NOHSO4 = H2S + 2 NO2 + 2 O2 + SO2,3.33e-113,11
7,4 NO + 2 NOHSO4 = H2S + 6 NO2 + SO2,2.25e-54,9
8,N2 + 8 NOHSO4 = 4 H2O + 10 NO2 + 8 SO2,4.85e-124,9
9,4 H2SO4 + N2 = 4 H2O + 2 NO2 + 4 SO2,3.54e-80,9
